# Load data

In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path

def load_data(path: str) -> pd.DataFrame:
    data = pd.read_csv(Path(path))
    return data

In [ ]:
data = load_data("../data/data.csv")

In [ ]:
data.info()

# Preprocessing

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
from sentence_transformers import SentenceTransformer

# intfloat/e5-small-v2 or intfloat/e5-base-v2 or intfloat/e5-large-v2
model = SentenceTransformer("intfloat/e5-small-v2")

In [ ]:
embeddings = model.encode(data['text'], normalize_embeddings=True, show_progress_bar=True)

In [ ]:
nb_comp = len(embeddings[0])
nb_data = len(embeddings)
print(f"embedding size : {nb_comp}")

In [ ]:
for num_comp in range(nb_comp):
    data['x_' + str(num_comp)] = [embeddings[k][num_comp] for k in range(nb_data)]

In [ ]:
data.head(10)

In [ ]:
data_keywords = data['keyword']

In [ ]:
data_keywords.size

In [ ]:
data_keywords.value_counts()

In [ ]:
columns = ['keyword']
data_dummy = pd.get_dummies(data, columns=columns, drop_first=True)
data_dummy.replace({False: 0, True: 1}, inplace=True)

In [ ]:
data_dummy.head(10)

In [ ]:
data_dummy.info()

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")
data_dummy_wNan = data_dummy.dropna()

def get_coordinates(location):
    try:
        location = geolocator.geocode(location)
        return location.latitude, location.longitude
    except:
        return None

data_tmp = data_dummy_wNan.head(10)
for location in data_tmp['location']:
    print(location, get_coordinates(location))
# data_tmp['coordinates'] = data1['location'].apply(get_coordinates)


In [ ]:
# data_dummy_wNan.apply(lambda x: get_coordinates(x['location']), axis=1)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# data_dummy_wNan = data_dummy.dropna()
# corr = data_dummy_wNan.corr()
# sns.heatmap(corr, annot=False, cmap='coolwarm', linewidths=0.5)


# Training

## K-NN

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# FIXME : use location
input = data_dummy.drop(columns=['text', 'target', 'location'])
output = data_dummy['target']

In [ ]:
input.head(5)

In [ ]:
input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5)

In [ ]:
clf.fit(input_train, output_train)
print(f"train accuracy : {clf.score(input_train, output_train)}")
print(f"test accuracy : {clf.score(input_test, output_test)}")

In [ ]:
grid_neighborns = {'n_neighbors' : [k for k in range(5, 100, 5)]}


In [ ]:
from tqdm import tqdm

training_accuracy = []
test_accuracy = []

for n_neighbors in tqdm(grid_neighborns):
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    clf.fit(input_train, output_train)
    training_accuracy.append(clf.score(input_train, output_train))
    test_accuracy.append(clf.score(input_test, output_test))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(grid_neighborns, training_accuracy, label="Training accuracy")
plt.plot(grid_neighborns, test_accuracy, label="Test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
grid_search = GridSearchCV(KNeighborsClassifier(), grid_neighborns, cv=10, n_jobs=-1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
grid_search.fit(input_train, output_train)

In [ ]:
print("Test set score: {:.2f}".format(grid_search.score(input_test, output_test)))

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))